In [ ]:
# https://www.insightbig.com/post/increasing-stock-returns-by-combining-williams-r-and-macd-in-python
# https://medium.com/codex/how-to-calculate-bollinger-bands-of-a-stock-with-python-f9f7d1184fc3 - Bollingerband
# Williams-r, MACD and Bollingerband trading strategy with backtesting

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import pandas_datareader as pdr
from math import floor
from termcolor import colored as cl

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (20,10)

In [3]:
symbol = 'AAPL'
df = pdr.DataReader(symbol, 'yahoo', '2021-01-01', '2021-12-31')
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-12-31,134.740005,131.720001,134.080002,132.690002,99116600.0,131.709213
2021-01-04,133.610001,126.760002,133.520004,129.410004,143301900.0,128.453430
2021-01-05,131.740005,128.429993,128.889999,131.009995,97664900.0,130.041611
2021-01-06,131.050003,126.379997,127.720001,126.599998,155088000.0,125.664207
2021-01-07,131.630005,127.860001,128.360001,130.919998,109578200.0,129.952286
...,...,...,...,...,...,...
2021-12-27,180.419998,177.070007,177.089996,180.330002,74919600.0,180.100540
2021-12-28,181.330002,178.529999,180.160004,179.289993,79144300.0,179.061859
2021-12-29,180.630005,178.139999,179.330002,179.380005,62348900.0,179.151749


In [4]:
# WILLIAMS %R CALCULATION

def get_wr(high, low, close, lookback):
    highh = high.rolling(lookback).max() 
    lowl = low.rolling(lookback).min()
    wr = -100 * ((highh - close) / (highh - lowl))
    return wr

df['wr_14'] = get_wr(df['High'], df['Low'], df['Close'], 14)
df.tail()

,High,Low,Open,Close,Volume,Adj Close,wr_14
Date,,,,,,,
2021-12-27,180.419998,177.070007,177.089996,180.330002,74919600.0,180.100540,-12.269961
2021-12-28,181.330002,178.529999,180.160004,179.289993,79144300.0,179.061859,-19.359318
2021-12-29,180.630005,178.139999,179.330002,179.380005,62348900.0,179.151749,-18.745742
2021-12-30,180.570007,178.089996,179.470001,178.199997,59773000.0,177.973251,-26.789423
2021-12-31,179.229996,177.259995,178.089996,177.570007,64062300.0,177.344055,-31.083832


In [7]:
# MACD CALCULATION

def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'Close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    return macd, signal, hist

df['macd'] = get_macd(df['Close'], 26, 12, 9)[0]
df['macd_signal'] = get_macd(df['Close'], 26, 12, 9)[1]
df['macd_hist'] = get_macd(df['Close'], 26, 12, 9)[2]
df = df.dropna()
df.tail()

,High,Low,Open,Close,Volume,Adj Close,wr_14,macd,macd_signal,macd_hist
Date,,,,,,,,,,
2021-12-27,180.419998,177.070007,177.089996,180.330002,74919600.0,180.100540,-12.269961,5.051209,5.174497,-0.123288
2021-12-28,181.330002,178.529999,180.160004,179.289993,79144300.0,179.061859,-19.359318,5.102919,5.160182,-0.057263
2021-12-29,180.630005,178.139999,179.330002,179.380005,62348900.0,179.151749,-18.745742,5.092459,5.146637,-0.054178
2021-12-30,180.570007,178.089996,179.470001,178.199997,59773000.0,177.973251,-26.789423,4.932099,5.103730,-0.171631
2021-12-31,179.229996,177.259995,178.089996,177.570007,64062300.0,177.344055,-31.083832,4.699999,5.022983,-0.322985


In [9]:
# BOLLINGER BANDS CALCULATION

def get_bollinger_bands(prices, rate=20):
    sma = prices.rolling(rate).mean()
    std = prices.rolling(rate).std()
    bollinger_up = sma + std * 2 # Calculate top band
    bollinger_down = sma - std * 2 # Calculate bottom band
    return bollinger_up, bollinger_down

bollinger_up, bollinger_down = get_bollinger_bands(df['Close'])
df['bollinger_up'] = bollinger_up
df['bollinger_down'] = bollinger_down
df = df.dropna()
df.tail()

C:\Users\eugen\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\eugen\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,High,Low,Open,Close,Volume,Adj Close,wr_14,macd,macd_signal,macd_hist,bollinger_up,bollinger_down
Date,,,,,,,,,,,,
2021-12-27,180.419998,177.070007,177.089996,180.330002,74919600.0,180.100540,-12.269961,5.051209,5.174497,-0.123288,183.586633,159.339368
2021-12-28,181.330002,178.529999,180.160004,179.289993,79144300.0,179.061859,-19.359318,5.102919,5.160182,-0.057263,183.797183,161.033817
2021-12-29,180.630005,178.139999,179.330002,179.380005,62348900.0,179.151749,-18.745742,5.092459,5.146637,-0.054178,184.389297,161.849703
2021-12-30,180.570007,178.089996,179.470001,178.199997,59773000.0,177.973251,-26.789423,4.932099,5.103730,-0.171631,184.555157,163.026842
2021-12-31,179.229996,177.259995,178.089996,177.570007,64062300.0,177.344055,-31.083832,4.699999,5.022983,-0.322985,184.263252,164.699748


In [13]:
# TRADING STRATEGY

def implement_wr_macd_bb_strategy(prices, wr, macd, macd_signal, bollinger_up, bollinger_down):    
    buy_price = []
    sell_price = []
    wr_macd_bb_signal = []
    signal = 0

    for i in range(len(wr)):
        if wr[i-1] > -50 and wr[i] < -50 and macd[i] > macd_signal[i] and bollinger_down[i] > prices[i]:
            if signal != 1:
                buy_price.append(prices[i])
                sell_price.append(np.nan)
                signal = 1
                wr_macd_bb_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_bb_signal.append(0)
                
        elif wr[i-1] < -50 and wr[i] > -50 and macd[i] < macd_signal[i] and bollinger_up[i] < prices[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(prices[i])
                signal = -1
                wr_macd_bb_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                wr_macd_bb_signal.append(0)
        
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            wr_macd_bb_signal.append(0)
            
    return buy_price, sell_price, wr_macd_bb_signal
            
buy_price, sell_price, wr_macd_bb_signal = implement_wr_macd_bb_strategy(df['Close'], df['wr_14'], df['macd'], df['macd_signal'], df['bollinger_up'], df['bollinger_down'])

In [14]:
# POSITION

position = []
for i in range(len(wr_macd_bb_signal)):
    if wr_macd_bb_signal[i] > 1:
        position.append(0)
    else:
        position.append(1)
        
for i in range(len(df['Close'])):
    if wr_macd_bb_signal[i] == 1:
        position[i] = 1
    elif wr_macd_bb_signal[i] == -1:
        position[i] = 0
    else:
        position[i] = position[i-1]
        
close_price = df['Close']
wr = df['wr_14']
macd_line = df['macd']
signal_line = df['macd_signal']
bollinger_up_line = df['bollinger_up']
bollinger_down_line = df['bollinger_down']
wr_macd_bb_signal = pd.DataFrame(wr_macd_bb_signal).rename(columns = {0:'wr_macd_bb_signal'}).set_index(df.index)
position = pd.DataFrame(position).rename(columns = {0:'wr_macd_bb_position'}).set_index(df.index)

frames = [close_price, wr, macd_line, signal_line, wr_macd_bb_signal, bollinger_up_line, bollinger_down_line, position]
strategy = pd.concat(frames, join = 'inner', axis = 1)

strategy.head()

,Close,wr_14,macd,macd_signal,wr_macd_bb_signal,bollinger_up,bollinger_down,wr_macd_bb_position
Date,,,,,,,,
2021-02-18,129.710007,-78.032447,-0.298319,0.528846,0,143.411642,128.878359,1
2021-02-19,129.869995,-76.504382,-0.594243,0.304228,0,143.571070,128.018931,1
2021-02-22,126.000000,-96.742660,-1.128037,0.017775,0,143.894179,126.388821,1
2021-02-23,125.860001,-61.672657,-1.544565,-0.294693,0,143.173539,125.403461,1
2021-02-24,125.349998,-64.289394,-1.893987,-0.614552,0,142.107547,124.688453,1


In [15]:
# BACKTESTING

df_ret = pd.DataFrame(np.diff(df['Close'])).rename(columns = {0:'returns'})
wr_macd_bb_strategy_ret = []

for i in range(len(df_ret)):
    try:
        returns = df_ret['returns'][i] * strategy['wr_macd_bb_position'][i]
        wr_macd_bb_strategy_ret.append(returns)
    except:
        pass
    
wr_macd_bb_strategy_ret_df = pd.DataFrame(wr_macd_bb_strategy_ret).rename(columns = {0:'wr_macd_bb_returns'})

investment_value = 100000
number_of_stocks = floor(investment_value / df['Close'][0])
wr_macd_bb_investment_ret = []

for i in range(len(wr_macd_bb_strategy_ret_df['wr_macd_bb_returns'])):
    returns = number_of_stocks * wr_macd_bb_strategy_ret_df['wr_macd_bb_returns'][i]
    wr_macd_bb_investment_ret.append(returns)

wr_macd_bb_investment_ret_df = pd.DataFrame(wr_macd_bb_investment_ret).rename(columns = {0:'investment_returns'})
total_investment_ret = round(sum(wr_macd_bb_investment_ret_df['investment_returns']), 2)
profit_percentage = floor((total_investment_ret / investment_value) * 100)
print(cl('Profit gained from the W%R MACD BB strategy by investing $100k in AAPL : {}'.format(total_investment_ret), attrs = ['bold']))
print(cl('Profit percentage of the W%R MACD BB strategy : {}%'.format(profit_percentage), attrs = ['bold']))

Profit gained from the W%R MACD BB strategy by investing $100k in AAPL : 36852.2
Profit percentage of the W%R MACD BB strategy : 36%


In [16]:
# SPY ETF COMPARISON

def get_benchmark(investment_value=100000):
    spy = pdr.DataReader('SPY', 'yahoo', '2021-01-01', '2021-12-31')['Close']
    benchmark = pd.DataFrame(np.diff(spy)).rename(columns = {0:'benchmark_returns'})
    
    number_of_stocks = floor(investment_value/spy[0])
    benchmark_investment_ret = []
    
    for i in range(len(benchmark['benchmark_returns'])):
        returns = number_of_stocks*benchmark['benchmark_returns'][i]
        benchmark_investment_ret.append(returns)

    benchmark_investment_ret_df = pd.DataFrame(benchmark_investment_ret).rename(columns = {0:'investment_returns'})
    return benchmark_investment_ret_df

benchmark = get_benchmark()
investment_value = 100000
total_benchmark_investment_ret = round(sum(benchmark['investment_returns']), 2)
benchmark_profit_percentage = floor((total_benchmark_investment_ret/investment_value)*100)
print(cl('Benchmark profit by investing $100k : {}'.format(total_benchmark_investment_ret), attrs = ['bold']))
print(cl('Benchmark Profit percentage : {}%'.format(benchmark_profit_percentage), attrs = ['bold']))
print(cl('W%R MACD BB Strategy profit is {}% higher than the Benchmark Profit'.format(profit_percentage - benchmark_profit_percentage), attrs = ['bold']))

Benchmark profit by investing $100k : 26988.36
Benchmark Profit percentage : 26%
W%R MACD BB Strategy profit is 10% higher than the Benchmark Profit
